Ukljucivanje potrebnih biblioteka i definisanje konstanti/parametara

In [165]:
%matplotlib notebook

import numpy as np
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import Audio

sample_freq = 10000

# PARAMETRI EQUALIZERA
F_tran = 50 # Sirina prelaznih opsega filtara
num_taps = 500 # Duzina filtara
Rc = 0.1 # Maksimalno dopusteno slabljenje u propusnom opsegu u decibelima
Rp = 55 # Minimalno dozvoljeno pojacanje u nepropusnom opsegu u decibelima
deltap = 1 - 10**(-Rc/20)
deltas = 10**(-Rp/20)
# Slijede parametri koji se ticu pojacanja po opsezima u decibelima
gain1 = 20
gain2 = 5
gain3 = 10
gain4 = -5
gain5 = -10

Generisanje slozenoperiodicnog signala i njegova reprodukcija

In [163]:
n = np.arange(0,3*sample_freq,1)
F = np.array([50, 120, 260, 440, 520, 720, 1020, 1250, 1400])
compound_sig = np.zeros([n.size])
i = 0
for freq in F:
    compound_sig += np.sin(2*np.pi*freq/sample_freq * n)
compound_sig /= np.max(np.abs(compound_sig))
Audio(compound_sig, rate = sample_freq)

Pravimo EQUALIZER efekat tako sto projektujemo banku od 5 filtara da pokrijemo opseg slozenoperiodicnog signala. Filtri imaju sledece granicne frekvencije (u Hz): (0,299) - (300,599) - (600, 899) - (900, 1199) - (1200,1500)

In [153]:
# Prvi filtar
Fc = 299
wc = 2*Fc/sample_freq 
wp = 2*(Fc+F_tran)/sample_freq
FILTER_1 = signal.remez(num_taps,[0,wc,wp,1], [1,0], [1/deltap, 1/deltas], fs = 2)

In [152]:
# Drugi filtar
Fc1 = 300
Fc2 = 599
wp1 = 2*(Fc1-F_tran)/sample_freq
wc1 = 2*Fc1/sample_freq 
wc2 = 2*Fc2/sample_freq
wp2 = 2*(Fc2+F_tran)/sample_freq
FILTER_2 = signal.remez(num_taps,[0,wp1,wc1,wc2,wp2,1], [0,1,0],[1/deltas, 1/deltap, 1/deltas], fs = 2)

In [151]:
# Treci filtar
Fc1 = 600
Fc2 = 899
wp1 = 2*(Fc1-F_tran)/sample_freq
wc1 = 2*Fc1/sample_freq 
wc2 = 2*Fc2/sample_freq
wp2 = 2*(Fc2+F_tran)/sample_freq
FILTER_3 = signal.remez(num_taps,[0,wp1,wc1,wc2,wp2,1], [0,1,0],[1/deltas, 1/deltap, 1/deltas], fs = 2)

In [150]:
# Cetvrti filtar
Fc1 = 900
Fc2 = 1199
wp1 = 2*(Fc1-F_tran)/sample_freq
wc1 = 2*Fc1/sample_freq 
wc2 = 2*Fc2/sample_freq
wp2 = 2*(Fc2+F_tran)/sample_freq
FILTER_4 = signal.remez(num_taps,[0,wp1,wc1,wc2,wp2,1], [0,1,0],[1/deltas, 1/deltap, 1/deltas], fs = 2)

In [164]:
# Peti filtar
Fc1 = 1200
Fc2 = 1499
wp1 = 2*(Fc1-F_tran)/sample_freq
wc1 = 2*Fc1/sample_freq 
wc2 = 2*Fc2/sample_freq
wp2 = 2*(Fc2+F_tran)/sample_freq
FILTER_5 = signal.remez(num_taps,[0,wp1,wc1,wc2,wp2,1], [0,1,0],[1/deltas, 1/deltap, 1/deltas], fs = 2)

Nakon sto smo projektovali filtre, izdvajamo opsege i pojacavamo specificiranim pojacanjima

In [172]:
band1 = signal.lfilter(FILTER_1, 1, compound_sig) * 10**(gain1/20)
Audio(band1, rate = sample_freq)